# Running Footprint Code

## Import Libraries

In [1]:
from pathlib import Path

import os

import numpy as np
import pandas as pd
import rasterio
import matplotlib.pyplot as plt
import pyproj
from fluxdataqaqc import Data
from rasterio.plot import show
import logging
import sys
sys.path.append("../../src/")
import fluxfootprints
from fluxfootprints import FFPModel as ffp 

#import utm
%matplotlib inline

c:\Users\paulinkenbrandt\Documents\GitHub\footprints\docs\notebooks\../../src\fluxfootprints\improved_ffp.py:1607: SyntaxWarning: invalid escape sequence '\s'
  """


In [2]:
stat_data = Data('US-UTE.ini')
stat_data.out_dir = Path('../../data/station_config/output')
stat_data.variables['MO_LENGTH'] = 'MO_LENGTH'
stat_data.variables['USTAR'] = 'USTAR'
stat_data.variables['V_SIGMA'] = 'V_SIGMA'
df = stat_data.df.rename(columns=stat_data.inv_map)
df = df.resample('h').mean()

Calculating mean for var: THETA from columns: ['SWC_1_1_1', 'SWC_2_1_1']
Converting vpd from hpa to kpa
Calculating vapor pressure from vapor pressure deficit and air temperature
Calculating dew point temperature from vapor pressure


c:\Users\paulinkenbrandt\.conda\envs\py313\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [3]:
latitude = stat_data.latitude
longitude = stat_data.longitude
station_coord = (latitude, longitude)
# get EPSG code from lat,long, convert to UTM https://epsg.io/32617
#EPSG=32700-np.round((45+latitude)/90.0)*100+np.round((-183+longitude)/6.0)
EPSG = 5070
#utm_coords = utm.from_latlon(latitude, longitude)

transformer = pyproj.Transformer.from_crs("EPSG:4326", f"EPSG:{EPSG:.0f}")
(station_y, station_x) = transformer.transform(*station_coord)
station = stat_data.site_id
(station_y, station_x)

(-1353156.22180492, 1744260.286299747)

In [4]:
print(latitude, longitude)

37.7353 -111.5708


In [6]:
#Other model parameters
h_c = 0.2 #Height of canopy [m]
# Estimated displacement height [m]
d = 10**(0.979* np.log10(h_c) - 0.154)

zm_s = 2. #Measurement height [m] from AMF metadata
h_s = 2000. #Height of atmos. boundary layer [m] - assumed
dx = 3. #Model resolution [m]
origin_d = 200. #Model bounds distance from origin [m]
# from 7 AM to 8 PM only, modify if needed
start_hr = 6
end_hr = 18
hours = np.arange(start_hr,end_hr+1)

#Loop through each day in the dataframe
for date in df.index.date[:10]:
    
    #Subset dataframe to only values in day of year
    print(f'Date: {date}')
    temp_df = df[(df.index.date == date)&(df.index.hour.isin(hours))]
        
    new_dat = None
    
    for indx,t in enumerate(hours):
        
        band = indx + 1
        print(f'Hour: {t}')

        try:
            temp_line = temp_df.loc[temp_df.index.hour == t,:]

            #Calculate footprint
            ffp_out = ffp(temp_line, 
                          domain=[-origin_d,origin_d,-origin_d,origin_d],
                          dx=dx,dy=dx,
                          zm=zm_s-d, 
                          h=h_s, 
                          rs=[1]*len(temp_line), 
                          z0=h_c*.123,
                          ol=temp_line['MO_LENGTH'].values,
                          sigmav=temp_line['V_SIGMA'].values,
                          ustar=temp_line['USTAR'].values, #umean=temp_line['ws'].values,
                          wind_dir=temp_line['wd'].values,
                          crop=0,
                          fig=0,
                          verbosity=2)
            ####verbosoity=2 prints out errors; if z0 triggers errors, use umean
            #    print(zm_s-d)
            temp_ffp = ffp_out.run()
            f_2d = np.array(temp_ffp['fclim_2d'])
            x_2d = np.array(temp_ffp['x_2d']) + station_x
            y_2d = np.array(temp_ffp['y_2d']) + station_y
            f_2d = f_2d*dx**2

            #Calculate affine transform for given x_2d and y_2d
            affine_transform = ffp.find_transform(y_2d,x_2d)

            #Create data file if not already created
            if new_dat is None:
                out_f = f"../../data/station_config/output/a{date}_{station}.tif"
                #out_f = f'../../station_config/output/a{date}_{station}.tif'
                print(f_2d.shape)
                new_dat = rasterio.open(out_f,'w',driver='GTiff',dtype=rasterio.float64,
                                        count=len(hours),height=f_2d.shape[0],width=f_2d.shape[1],
                                        transform=affine_transform,
                                        crs=transformer.target_crs,
                                        nodata=0.00000000e+000)

        except Exception as e:

            print(f'Hour {t} footprint failed, band {band} not written.')
            print(e)
            temp_ffp = None

            continue

        #Mask out points that are below a % threshold (defaults to 90%)
        f_2d = ffp.mask_fp_cutoff(f_2d)

        #Write the new band
        new_dat.write(f_2d,indx+1)

        #Update tags with metadata
        tag_dict = {'hour':f'{t*100:04}',
                    'wind_dir':temp_line['wd'].values,
                    'total_footprint':np.nansum(f_2d)}

        new_dat.update_tags(indx+1,**tag_dict)
    
    #Close dataset if it exists
    try:
        new_dat.close()
    except:
        continue
    
    print()
    


2025-05-07 13:35:07,193 - FFPModel - DEBUG - Valid input length: 0
2025-05-07 13:35:07,196 - FFPModel - INFO - Mean RSL height (z*): nanm, Measurement height: 2.0m
2025-05-07 13:35:07,196 - FFPModel - INFO - Setting up computational domain...
2025-05-07 13:35:07,197 - FFPModel - DEBUG - Domain dimensions - x: 135, y: 135
2025-05-07 13:35:07,201 - FFPModel - DEBUG - Grid shapes - xv: (135, 135), rho: (135, 135)


2025-05-07 13:35:07,202 - FFPModel - INFO - Domain setup completed successfully
c:\Users\paulinkenbrandt\Documents\GitHub\footprints\docs\notebooks\../../src\fluxfootprints\improved_ffp.py:762: RuntimeWarning: invalid value encountered in divide
  percentage = (count / self.ts_len) * 100
2025-05-07 13:35:07,242 - FFPModel - INFO - strongly_unstable: 0 points (nan%)
c:\Users\paulinkenbrandt\Documents\GitHub\footprints\docs\notebooks\../../src\fluxfootprints\improved_ffp.py:762: RuntimeWarning: invalid value encountered in divide
  percentage = (count / self.ts_len) * 100
2025-05-07 13:35:07,243 - FFPModel - INFO - unstable: 0 points (nan%)
c:\Users\paulinkenbrandt\Documents\GitHub\footprints\docs\notebooks\../../src\fluxfootprints\improved_ffp.py:762: RuntimeWarning: invalid value encountered in divide
  percentage = (count / self.ts_len) * 100
2025-05-07 13:35:07,244 - FFPModel - INFO - neutral: 0 points (nan%)
c:\Users\paulinkenbrandt\Documents\GitHub\footprints\docs\notebooks\../../s

Date: 2024-06-24
Hour: 6
Hour 6 footprint failed, band 1 not written.
zero-size array to reduction operation fmin which has no identity
Hour: 7
Hour 7 footprint failed, band 2 not written.
zero-size array to reduction operation fmin which has no identity
Hour: 8


c:\Users\paulinkenbrandt\Documents\GitHub\footprints\docs\notebooks\../../src\fluxfootprints\improved_ffp.py:762: RuntimeWarning: invalid value encountered in divide
  percentage = (count / self.ts_len) * 100
2025-05-07 13:35:07,413 - FFPModel - INFO - strongly_unstable: 0 points (nan%)
c:\Users\paulinkenbrandt\Documents\GitHub\footprints\docs\notebooks\../../src\fluxfootprints\improved_ffp.py:762: RuntimeWarning: invalid value encountered in divide
  percentage = (count / self.ts_len) * 100
2025-05-07 13:35:07,414 - FFPModel - INFO - unstable: 0 points (nan%)
c:\Users\paulinkenbrandt\Documents\GitHub\footprints\docs\notebooks\../../src\fluxfootprints\improved_ffp.py:762: RuntimeWarning: invalid value encountered in divide
  percentage = (count / self.ts_len) * 100
2025-05-07 13:35:07,415 - FFPModel - INFO - neutral: 0 points (nan%)
c:\Users\paulinkenbrandt\Documents\GitHub\footprints\docs\notebooks\../../src\fluxfootprints\improved_ffp.py:762: RuntimeWarning: invalid value encountered

Hour 8 footprint failed, band 3 not written.
zero-size array to reduction operation fmin which has no identity
Hour: 9
Hour 9 footprint failed, band 4 not written.
zero-size array to reduction operation fmin which has no identity
Hour: 10
Hour 10 footprint failed, band 5 not written.
zero-size array to reduction operation fmin which has no identity
Hour: 11


...

Hour 7 footprint failed, band 2 not written.
zero-size array to reduction operation fmin which has no identity
Hour: 8
Hour 8 footprint failed, band 3 not written.
zero-size array to reduction operation fmin which has no identity
Hour: 9
Hour 9 footprint failed, band 4 not written.
zero-size array to reduction operation fmin which has no identity
Hour: 10


2025-05-07 13:35:13,043 - FFPModel - DEBUG - Traceback:
Traceback (most recent call last):
  File "c:\Users\paulinkenbrandt\Documents\GitHub\footprints\docs\notebooks\../../src\fluxfootprints\improved_ffp.py", line 1058, in calc_xr_footprint
    f"xstar_ci_dummy range: {float(xstar_ci_dummy.min())} to {float(xstar_ci_dummy.max())}"
                                   ~~~~~~~~~~~~~~~~~~^^
  File "c:\Users\paulinkenbrandt\.conda\envs\py313\Lib\site-packages\xarray\core\_aggregations.py", line 2899, in min
    return self.reduce(
           ~~~~~~~~~~~^
        duck_array_ops.min,
        ^^^^^^^^^^^^^^^^^^^
    ...<3 lines>...
        **kwargs,
        ^^^^^^^^^
    )
    ^
  File "c:\Users\paulinkenbrandt\.conda\envs\py313\Lib\site-packages\xarray\core\dataarray.py", line 3875, in reduce
    var = self.variable.reduce(func, dim, axis, keep_attrs, keepdims, **kwargs)
  File "c:\Users\paulinkenbrandt\.conda\envs\py313\Lib\site-packages\xarray\core\variable.py", line 1654, in reduce
    res

Hour 10 footprint failed, band 5 not written.
zero-size array to reduction operation fmin which has no identity
Hour: 11
Hour 11 footprint failed, band 6 not written.
zero-size array to reduction operation fmin which has no identity
Hour: 12
Hour 12 footprint failed, band 7 not written.
zero-size array to reduction operation fmin which has no identity
Hour: 13
Hour 13 footprint failed, band 8 not written.
zero-size array to reduction operation fmin which has no identity
Hour: 14


2025-05-07 13:35:13,255 - FFPModel - INFO - Mean RSL height (z*): 0.7m, Measurement height: 2.0m
2025-05-07 13:35:13,261 - FFPModel - DEBUG - Valid input length: 1
2025-05-07 13:35:13,263 - FFPModel - INFO - Mean RSL height (z*): 0.7m, Measurement height: 2.0m
2025-05-07 13:35:13,269 - FFPModel - DEBUG - Valid input length: 1
2025-05-07 13:35:13,271 - FFPModel - INFO - Mean RSL height (z*): 0.7m, Measurement height: 2.0m
2025-05-07 13:35:13,277 - FFPModel - DEBUG - Valid input length: 1
2025-05-07 13:35:13,279 - FFPModel - INFO - Mean RSL height (z*): 0.7m, Measurement height: 2.0m
2025-05-07 13:35:13,285 - FFPModel - DEBUG - Valid input length: 1
2025-05-07 13:35:13,289 - FFPModel - INFO - Mean RSL height (z*): 0.7m, Measurement height: 2.0m


Hour 14 footprint failed, band 9 not written.
all rs values must be between 0 and 1
Hour: 15
Hour 15 footprint failed, band 10 not written.
all rs values must be between 0 and 1
Hour: 16
Hour 16 footprint failed, band 11 not written.
all rs values must be between 0 and 1
Hour: 17
Hour 17 footprint failed, band 12 not written.
all rs values must be between 0 and 1
Hour: 18
Hour 18 footprint failed, band 13 not written.
all rs values must be between 0 and 1


In [ ]:

temp_line

In [1]:
plt.figure(figsize=(14,14))
src = rasterio.open("../../station_config/output/a2021-05-20_US-UTE.tif")
show(src)

NameError: name 'plt' is not defined

In [ ]:
show(src.read(6))
# Daily average
arrs = []
cnt = 0
for i in range(1, 15):
    arr = src.read(i)
    if (arr == 0).all():
        continue
    else:
        cnt += 1
        arrs.append(arr)
avg_for_day = sum(arrs) / cnt

plt.figure(figsize=(14, 14))
plt.imshow(avg_for_day)